In [4]:
## notebook to get advancement decisions across years.
# depends on DenodoConnection(), which may not be widely available
#%pip install psycopg2-binary

  Using cached psycopg2_binary-2.9.9-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.4 kB)
Using cached psycopg2_binary-2.9.9-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
# import packages
import os
import sys
import pandas as pd
import numpy as np

# need to add dme_sagemaker to path to load in libraries
sys.path.append("/root/dme_sagemaker/dme_sagemaker")

from libs.denodo.denodo_connection import DenodoConnection

In [2]:
# set vars:
ap_data_sector = 'CORN_NA_SUMMER'
analysis_year = 2022

In [ ]:
# get maximum stage per material per year via SQL
def get_material_by_trialstage_year_one_sector(ap_data_sector, min_year=2012, max_year=2025):
    query_str = """
        WITH "data_sectors" as (
            select distinct sector1.ap_data_sector_name,
                sector1.spirit_country_code_list,
                sector1.spirit_crop_name, 
                sector1.spirit_crop_guid, 
                sector1.spirit_season_code_list
            from managed.rv_ap_data_sector_config sector1
            inner join managed.rv_ap_data_sector_config sector2
            on sector1.spirit_country_code_list = sector2.spirit_country_code_list 
                and sector1.spirit_crop_name = sector2.spirit_crop_name 
            where sector2.ap_data_sector_name = {0}

        )
            SELECT
            "data_sectors"."ap_data_sector_name",
            "rv_bb_experiment_trial_entry_sdl"."year" AS "year",
            "rv_bb_experiment_trial_entry_sdl"."be_bid" AS "be_bid",
        --  "rv_be_bid_ancestry_laas"."receiver_p" AS "fp_be_bid",
        --  "rv_be_bid_ancestry_laas"."donor_p" AS "mp_be_bid",
            COALESCE("rv_corn_material_tester"."fp_be_bid", "rv_be_bid_ancestry_laas"."receiver_p") AS "fp_be_bid",
            COALESCE("rv_corn_material_tester"."mp_be_bid", "rv_be_bid_ancestry_laas"."donor_p") AS "mp_be_bid",
            "rv_bb_experiment_trial_entry_sdl"."season" AS "season",
            MAX("rv_bb_experiment_trial_entry_sdl"."stage_lid") AS "stage_lid"
          FROM (
            SELECT 
              CASE 
                  WHEN "cpi" > 0
                      THEN 7
                  WHEN CAST("stage_lid" AS decimal) < 7 AND CAST("stage_lid" AS decimal) > 0
                      THEN CAST(ROUND(CAST("stage_lid" AS decimal), 0) AS integer)
                  WHEN LOWER("trialcore_descr") LIKE '% stage%'
                      THEN CAST(COALESCE(99, SUBSTRING(LOWER("trialcore_descr"), INSTR(LOWER("trialcore_descr"),'stage') + LEN('stage'), INSTR(LOWER("trialcore_descr"),'stage') + LEN('stage')+1)) AS integer)
                  WHEN LOWER("trialcore_descr") LIKE '% stg%'
                      THEN CAST(COALESCE(99, SUBSTRING(LOWER("trialcore_descr"), INSTR(LOWER("trialcore_descr"),'stg') + LEN('stg'), INSTR(LOWER("trialcore_descr"),'stage') + LEN('stg')+1)) AS integer)
                  WHEN "experiment_trial_no" LIKE '[1-6]%'
                      THEN CAST(SUBSTRING("experiment_trial_no",1,1) AS integer)
                  ELSE 99
              END AS "stage_lid",
              CAST("year" AS integer) AS "year",
              "crop_guid",
              "cpi",
              "material_guid",
              "be_bid",
              "season",
              "crop_code",
              "location_guid",
              "trial_guid",
              "planting_area_guid",
              "breeding_group_guid",
              "trial_experiment_guid"
              FROM "managed"."rv_bb_experiment_trial_entry_sdl"
              WHERE "year" >= {1}
                AND ("stage_lid" IS NOT NULL AND "stage_lid" != '')
                AND "year" < {2}
                AND "be_bid" IS NOT NULL
                AND "experiment_id" IS NOT NULL
            ) "rv_bb_experiment_trial_entry_sdl"
          INNER JOIN "data_sectors"
            ON "rv_bb_experiment_trial_entry_sdl"."crop_guid" = "data_sectors"."spirit_crop_guid"
            AND "data_sectors"."spirit_season_code_list" LIKE CONCAT('%', "rv_bb_experiment_trial_entry_sdl"."season", '%')
          INNER JOIN (
            SELECT DISTINCT
              "location_guid",
              "crop_guid",
              "country_code",
              "region_lid"
              FROM "managed"."rv_bb_location_sdl"
              WHERE "region_lid" IN ('NOAM','EAME','APAC','LATAM','CHINA')
          ) "rv_bb_location_sdl"
            ON ("rv_bb_experiment_trial_entry_sdl"."location_guid" = "rv_bb_location_sdl"."location_guid")
              AND ("rv_bb_experiment_trial_entry_sdl"."crop_guid" = "rv_bb_location_sdl"."crop_guid")
              AND "data_sectors"."spirit_country_code_list" LIKE CONCAT('%', "rv_bb_location_sdl"."country_code", '%')
        LEFT JOIN "advancement"."rv_corn_material_tester" "rv_corn_material_tester"
            ON "rv_bb_experiment_trial_entry_sdl"."be_bid" = "rv_corn_material_tester"."be_bid"
            AND "data_sectors"."ap_data_sector_name" LIKE 'CORN%'
        LEFT JOIN "managed"."rv_be_bid_ancestry_laas" "rv_be_bid_ancestry_laas"
            ON "rv_bb_experiment_trial_entry_sdl"."be_bid" = "rv_be_bid_ancestry_laas"."be_bid"
        WHERE "rv_bb_experiment_trial_entry_sdl"."stage_lid" <= 7
        GROUP BY
            "data_sectors"."ap_data_sector_name",
            "rv_bb_experiment_trial_entry_sdl"."year",
            "rv_bb_experiment_trial_entry_sdl"."crop_guid",
            "rv_bb_experiment_trial_entry_sdl"."be_bid",
            "rv_bb_experiment_trial_entry_sdl"."season",
            "rv_corn_material_tester"."fp_be_bid",
            "rv_be_bid_ancestry_laas"."receiver_p",
            "rv_corn_material_tester"."mp_be_bid", 
            "rv_be_bid_ancestry_laas"."donor_p"

    """.format("'" + ap_data_sector + "'", int(min_year), int(max_year))

    with DenodoConnection() as dc:
        output_df = dc.get_data(query_str)
    return output_df
    
# get data per year, concat across year. This should reduce load compared to getting data from all 5 years simultaneously.
df_get_bebid_list=[]
for yr in range(int(analysis_year)-2, int(analysis_year)+3,1):
    df_get_bebid_list.append(get_material_by_trialstage_year_one_sector(
        ap_data_sector=ap_data_sector,
        min_year=yr,
        max_year=yr+1
    ))

df_get_bebid = pd.concat(df_get_bebid_list,axis=0).drop_duplicates()
print(df_get_bebid.shape)

/root/dme_sagemaker/dme_sagemaker/libs/denodo/denodo_connection.py:61: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, self.__denodo_con)


In [ ]:
# load historical advancement decisions
# preprocess SQL query:
# stack parents and enties to process both
# pivot across years, get maximum stage per material
def get_bebid_advancement_decisions(df, input_year):
    # get be_bid, and parental be_bids, plus stage tested from denodo
    ##df = pd.read_csv(os.path.join(folder, 'data_'+input_year+'_get_bebid.csv'))

    # stack be_bid, fp_be_bid, mp_be_bid, create material_type column. This lets us do both entries and parents simultaneously
    df_stack = stack_bebids(df)
    df_stack = df_stack.drop_duplicates()

    yrs = pd.unique(df_stack['year'])
    yrs = np.sort(yrs)

    # pivot dataframe by year and season
    df_stack_piv = df_stack.pivot_table(values=['stage'],
                                        index=['ap_data_sector_name', 'season', 'be_bid',
                                               'material_type'],
                                        columns=['year'],
                                        aggfunc='max').reset_index()
    # update column names
    df_stack_piv.columns = df_stack_piv.columns.map(
        lambda x: '_'.join([str(i) for i in x]).replace('value', '').strip('_'))

    # check for bebid aliases via abbr_code
    df_stack_piv = get_bebid_aliases(df_stack_piv)

    # rename columns
    stage_cols = []
    for col in df_stack_piv.columns:
        if 'stage_' in col:
            stage_cols.append(col)

    # get stage max column
    df_stack_piv['stage_max'] = df_stack_piv[stage_cols].max(axis=1)

    # clean year-stage info
    for yr in yrs[1:]:
        # if current year is null
        # ->if year prior is not null
        curr_stage_col = 'stage_' + str(yr)
        prev_stage_col = 'stage_' + str(yr - 1)
        future_stage_cols = ['stage_' + str(yr_next) for yr_next in
                             range(yr + 1, np.minimum(yr + 5, yrs[-1]) + 1)]

        # ->if all of future years are null, set as = 13
        # else = stage from prev year
        adj_mask = (df_stack_piv[curr_stage_col].isna()) & (df_stack_piv[prev_stage_col].notna())
        future_mask = np.all(df_stack_piv[future_stage_cols].isna(), axis=1)

        df_stack_piv[curr_stage_col][(adj_mask) & (future_mask)] = 13
        df_stack_piv[curr_stage_col][(adj_mask) & (future_mask == False)] = df_stack_piv[prev_stage_col][
            (adj_mask) & (future_mask == False)]

    # make stage achieved columns, fill with 0 to start
    stages = [1, 2, 3, 4, 5, 6, 7]

    for stage in stages:
        if stage < 7:
            stage_name = 'stage_' + str(stage)
        else:
            stage_name = 'stage_chk'
        df_stack_piv[stage_name] = np.any(df_stack_piv[stage_cols].values == stage, axis=1).astype(int)

    return df_stack_piv

# stack be_bid, parental be_bids into one dataframe to process simultaneously. Make material_type column to indicate parent/entry
def stack_bebids(df_in):
    df_list = []
    for col in ['be_bid', 'fp_be_bid', 'mp_be_bid']:
        df_temp = df_in[['ap_data_sector_name', 'year', 'season', col, 'stage_lid']].rename(
            columns={col: 'be_bid','stage_lid':'stage'})
        df_temp = df_temp[(df_temp['be_bid'].notna()) & (df_temp['be_bid'] != '')]
        if col == 'be_bid':
            df_temp['material_type'] = 'entry'
        else:
            df_temp['material_type'] = 'parent'
        df_list.append(df_temp)

    df_stack = pd.concat(df_list, axis=0)
    return df_stack

# check for be_bids that share the same abbr_code. This cleans up some advancement decisions in corn_brazil
def get_bebid_aliases(df_in, be_bid_col='be_bid',meta_cols=['ap_data_sector_name', 'season', 'be_bid', 'material_type']):
    # we need to create an alias map between be_bids that share the same abbr code....
    abbr_code_list = []
    step = 1000
    bebids = df_in[be_bid_col].drop_duplicates()
    with DenodoConnection() as dc:
        for i in range(0, bebids.shape[0], step):
            if i + step >= bebids.shape[0]:
                bebids_step = bebids.iloc[i:]
            else:
                bebids_step = bebids.iloc[i:i + step]

            bebids_query = make_search_str(bebids_step)

            query_str = """
                select distinct be_bid, abbr_code 
                from managed.rv_bb_material_sdl
                where be_bid in {0}
            """.format(bebids_query)
            abbr_code_list.append(dc.get_data(query_str))

    df_abbr = pd.concat(abbr_code_list, axis=0)

    # get bebid map
    df_bebid_map = df_abbr.merge(df_abbr, on=['abbr_code'], how='inner', suffixes=('1', '2'))
    df_bebid_map = df_bebid_map[df_bebid_map['be_bid1'] != df_bebid_map['be_bid2']]

    # convert bebids if same abbr code
    df_out = df_in.copy()
    for check_col, other_col in zip(['be_bid1', 'be_bid2'], ['be_bid2', 'be_bid1']):
        df_temp = df_out.merge(df_bebid_map[[check_col, other_col]], left_on=[be_bid_col], right_on=[check_col],
                                     how='left')
        df_temp = df_temp[df_temp[other_col].notna()]
        df_temp = df_temp.drop(columns=[be_bid_col, check_col]).rename(columns={other_col: be_bid_col})

        df_out = pd.concat((df_out, df_temp), axis=0).groupby(
            by=meta_cols).max().reset_index()
    df_out = df_out.drop_duplicates()

    return df_out

# given a list of strings, generate SQL string to check if var is in that list of strings
# SQL Code: var in (search_ids[0], search_ids[1]....)
# currently handles ' within search ids...other escape characters may cause issues
def make_search_str(search_ids):
    search_id_str = '('
    # get unique list of source ids
    uniq_search_ids = pd.unique(search_ids)

    for search_id in uniq_search_ids:
        if isinstance(search_id, str):
            if "'" in search_id: # put \' in string...
                search_id_split = search_id.split("'")
                
                search_id_str = search_id_str + "'"
                
                for s in search_id_split:
                    search_id_str = search_id_str + s + "\\" + "'"
            
                search_id_str = search_id_str[:-2] + "'," # remove last addition|
            else:
                search_id_str = search_id_str + "'" + search_id + "',"
                
    # remove last comma, then close parantheses
    search_id_str = search_id_str[:-1] + ')'
    return search_id_str


df_stage_piv = get_bebid_advancement_decisions(
    df=df_get_bebid,
    input_year=yr,
)

In [ ]:
df_stage_piv.head()

In [ ]:
df_stage_piv[(df_stage_piv['stage_2023'] == 4) & (df_stage_piv['stage_2024'] < 7) & (df_stage_piv['stage_2024'] >= 4)]